In [1]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
import itertools
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
%matplotlib inline

C:\Users\Bjorn\anaconda3\envs\new_tf\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Bjorn\anaconda3\envs\new_tf\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
# Import the ratings dataset
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [3]:
new_ratings = ratings.iloc[np.isin(ratings["movieId"],ratings["movieId"].unique()[:5] )]

In [4]:
user_movie = pd.crosstab(new_ratings.userId,new_ratings.movieId, values=new_ratings.rating, aggfunc="mean")

In [5]:
user_movie_train, user_movie_test = train_test_split(user_movie, test_size=0.3)

In [6]:
from sklearn.cluster import KMeans
elbow = [] 
for i in range(2, 10):
    kmeans = make_pipeline(KNNImputer(n_neighbors=3),KMeans(n_clusters = i, init = 'k-means++', random_state = 42))
    #kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(user_movie_train) 
    elbow.append(kmeans.named_steps['kmeans'].inertia_)
    print("Kmeans med {} klynger".format(i))

Kmeans med 2 klynger
Kmeans med 3 klynger


MemoryError: Unable to allocate 1.00 GiB for an array with shape (1789, 75000) and data type float64

In [ ]:
plt.figure()
plt.plot(elbow)
plt.xticks(np.arange(0,14),np.arange(1,15))
plt.show()

In [ ]:
kmeans =  make_pipeline(KNNImputer(n_neighbors=3),KMeans(n_clusters = 12, init = 'k-means++', random_state = 42))
kmeans.fit(user_movie_train)

In [ ]:
y_user_movie_hat = kmeans.predict(user_movie_test)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42) # use fit_transform instead of fit, as TSNE has no transform method 
movie_tsne = tsne.fit_transform(kmeans["knnimputer"].transform(user_movie_train))

In [ ]:
for i in sorted(np.unique(y_user_movie_hat)):
    temp_scatter = movie_tsne[np.where(y_user_movie_hat==i)]
    plt.scatter(x=temp_scatter[:,0],y=temp_scatter[:,1], color="C{}".format(i))
plt.legend()
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca =  PCA(n_components=2)

movie_pca = pca.fit_transform(kmeans["knnimputer"].transform(user_movie_train))

In [ ]:
for i in sorted(np.unique(y_user_movie_hat)):
    temp_scatter = movie_pca[np.where(y_user_movie_hat==i)]
    plt.scatter(x=temp_scatter[:,0],y=temp_scatter[:,1], color="C{}".format(i))
plt.show()

### Designe anbefalingssystem

In [ ]:
test_not_nan = user_movie_test.loc[user_movie_test.isnull().any(axis=1) == False]

In [ ]:
class abonnent():
    def __init__(self, fasit):
        self.fasit = fasit
        self.idx1 = None
        self.filmer_sett = np.ones((1,5))*np.nan
    
    def start(self):
        self.idx1 = np.random.choice(len(self.fasit[0]))
        self.filmer_sett[0][self.idx1] = self.fasit[0][self.idx1]
        return self.filmer_sett
    
    def ny_film(self, anbefaling):
        if np.isin(anbefaling, np.where(np.isnan(self.filmer_sett))[1]) == True:
            self.filmer_sett[0][anbefaling] = self.fasit[0][anbefaling]
            print("Abonnentens rating = ", self.fasit[0][anbefaling])
            return self.filmer_sett
        else:
            print("denne filmen har jeg sett!")
            return self.filmer_sett

In [ ]:
class anbefalingssystem():
    def __init__(self, modell):
        self.model = modell
        self.pred = []
        #self.filmer_sett = np.ones((1,5))
    
    def anbefaling(self, rating):
        pred_class = self.model.predict(rating)
        self.pred.append(pred_class[0])
        cluster_ratings = self.model["kmeans"].cluster_centers_[pred_class]
        sett_idx = np.where(~np.isnan(rating))[1]
        cluster_ratings[0][sett_idx] = 0
        new_movie = np.argmax(cluster_ratings)
        print("Anbefaler film nr.", new_movie)
        print("Predikert rating = ", cluster_ratings.max())
        return new_movie

In [ ]:
for i in range(len(test_not_nan)):
    abo = abonnent(np.asarray([test_not_nan.iloc[i]]))
    anbef = anbefalingssystem(kmeans)
    film_array = abo.start()
    for i in range(4):
        ny_film = anbef.anbefaling(film_array)
        film = abo.ny_film(ny_film)
    print(anbef.pred)
    print("----------------")
    